In [1]:
import torch
from torch import nn
from collections import OrderedDict
import numpy as np

In [4]:
class CoDA_Regress(nn.Module):
    
    def __init__(self, encoder_shape, decoder_shape):
        super(CoDA_AE, self).__init__()
        
        #define regression layer, with input dim based on final encoding layer
        self.dim = endcoder_shape[-1]
        self.linear = nn.Linear(self.dim, 1)
        
        #define a list with each element the input and output dim of the layer
        layer_list = [[encoder_shape[i], encoder_shape[i+1]] for i in range(0, len(encoder_shape)-1)]
        
        encoder_dict = OrderedDict()

        for i in range(0,len(layer_list)):
            encoder_dict["layer"  + str(i)] = nn.ELU(nn.Linear(layer_list[i][0], layer_list[i][1]))

        self.encoder = nn.Sequential(encoder_dict)
        
        layer_list = [[decoder_shape[i], decoder_shape[i+1]] for i in range(0, len(decoder_shape)-1)]
        
        decoder_dict = OrderedDict()

        for i in range(0,len(layer_list)):
            decoder_dict["layer"  + str(i)] = nn.ELU(nn.Linear(layer_list[i][0], layer_list[i][1]))

        self.decoder = nn.Sequential(decoder_dict)
        
        
    def forward(x):
        #run the encoding and store the low level representation as A
        self.A = self.encoder(x)
        self.reconstruction = self.decoder(self.A)
        self.y = self.linear(self.A)
        #return both the predicted target, and the reconstruction so both can be inputs to the combined loss
        return self.pred, self.reconstruction


In [2]:
class Combined_Loss(torch.nn.Module):
    def __init__(self, lam):
        super(Combined_Loss,self).__init__()
        self.CoDA_Loss = CoDA_Loss()
        self.MSE = nn.MSELoss()
        self.lambda = lam
        
    def forward(self,Y,X,y_hat,y):
        #X is original data, Y is CoDA reconstruction, y is targets, y_hat 
        #input needs to be normalised by g(x) (geometric mean) for X_hat
        #TODO centering matrix? Reduce mean? Mask?  
        return self.MSE(yhat, y) + lam * CoDA_Loss(Y,X)  

class CoDA_Loss(torch.nn.Module):
    def __init__(self):
        super(CoDA_Loss,self).__init__()
        
    def forward(self,Y,X):
        #X is original data, Y is CoDA reconstruction
        #input needs to be normalised by g(x) (geometric mean) for X_hat
        #TODO centering matrix? Reduce mean? Mask?  
        X_check = check(X)
        coda_loss = torch.sum(torch.exp(Y)) - torch.sum(X_check * Y)
        return coda_loss  
    
def check(X):
    #assume input is tensor so we can use the numpy() method
    assert type(X) == torch.Tensor
    gmean = torch.prod(X, 1) ** (1./X.shape[1])
    return torch.div(X.t(), gmean).t()